In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from sklearn.model_selection import train_test_split
import shutil

# Define Paths
data_dir = "./data/wider_face/images"  # Path to WIDER FACE images
train_dir = "./data/wider_face_train"  # Path to training data
val_dir = "./data/wider_face_val"      # Path to validation data
img_size = (128, 128)                  # Input image size
batch_size = 32                        # Batch size for training
epochs = 50                            # Number of epochs
num_classes = 61                       # Number of classes in WIDER FACE dataset

# Ensure train and val directories exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Manually split dataset if not already done
if not os.listdir(train_dir) or not os.listdir(val_dir):
    images = os.listdir(data_dir)
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)
    
    # Move images to train and val directories
    for img in train_images:
        shutil.copy(os.path.join(data_dir, img), os.path.join(train_dir, img))
    for img in val_images:
        shutil.copy(os.path.join(data_dir, img), os.path.join(val_dir, img))

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,                     # Normalize pixel values to [0, 1]
    rotation_range=10,                  # Randomly rotate images by 10 degrees
    width_shift_range=0.1,              # Randomly shift images horizontally by 10%
    height_shift_range=0.1,             # Randomly shift images vertically by 10%
    horizontal_flip=True                # Randomly flip images horizontally
)

val_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for validation data

# Load Data
train_data = train_datagen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
val_data = val_datagen.flow_from_directory(
    val_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')

# Load Pretrained ResNet50 Model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze initial layers
for layer in base_model.layers[:-50]:
    layer.trainable = False

# Add Custom Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling layer
x = Dense(512, activation='relu')(x)  # Fully connected layer
x = BatchNormalization()(x)  # Batch normalization
x = Dropout(0.5)(x)  # Dropout for regularization
x = Dense(256, activation='relu')(x)  # Additional fully connected layer
x = BatchNormalization()(x)  # Batch normalization
x = Dropout(0.5)(x)  # Dropout for regularization
out = Dense(num_classes, activation='softmax')(x)  # Output layer for multi-class classification

# Create the final model
model = Model(inputs=base_model.input, outputs=out)

# Compile Model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Learning Rate Scheduler
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

# Define Callbacks
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1),  # Reduce learning rate on plateau
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),  # Early stopping
    ModelCheckpoint("./models/face_detection_model.keras", save_best_only=True),  # Save the best model
    LearningRateScheduler(lr_scheduler)  # Learning rate scheduler
]

# Train Model
history = model.fit(train_data, validation_data=val_data, epochs=epochs, callbacks=callbacks)

# Evaluate Model
loss, accuracy = model.evaluate(val_data)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Found 2558 images belonging to 61 classes.
Found 668 images belonging to 61 classes.
Epoch 1/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.0136 - loss: 5.5065 - val_accuracy: 0.0090 - val_loss: 4.1791 - learning_rate: 1.0000e-04
Epoch 2/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 113s 1s/step - accuracy: 0.0233 - loss: 5.1501 - val_accuracy: 0.0539 - val_loss: 4.0941 - learning_rate: 1.0000e-04
Epoch 3/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1119s 14s/step - accuracy: 0.0341 - loss: 5.0760 - val_accuracy: 0.0344 - val_loss: 4.1873 - learning_rate: 1.0000e-04
Epoch 4/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 0.0347 - loss: 4.8921 - val_accuracy: 0.0868 - val_loss: 4.4188 - learning_rate: 1.0000e-04
Epoch 5/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.0411 - loss: 4.8253 - val_accuracy: 0.0269 - val_loss: 4.3335 - learning_rate: 1.0000e-04
Epoch 6/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.0418 - loss: 4.9142 - val_accuracy: 0.0554 - val_loss: 4.6844 - learning_rat

KeyboardInterrupt: 

In [3]:
import tensorflow as tf
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
